In [4]:
!pip install pygame

     ---------------------------------------- 10.6/10.6 MB 5.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install pygame pillow mutagen



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ------------------------------------ 194.4/194.4 kB 840.9 kB/s eta 0:00:00


In [10]:
import tkinter as tk
from tkinter import filedialog, Listbox, messagebox
from PIL import Image, ImageTk
from mutagen.id3 import ID3
from mutagen.mp3 import MP3
import io
import pygame
import os

# Initialize pygame
pygame.mixer.init()

# Tkinter window setup
root = tk.Tk()
root.title("MP3 Player")
root.geometry("500x700")
root.config(bg="lightblue")

# Global states
paused = False
looping = False
song_list = []
current_index = None
supported_formats = (".mp3", ".wav", ".ogg", ".flac")

# Default image area
img_label = tk.Label(root, bg="lightblue")
img_label.pack(pady=10)

# Status label
status_label = tk.Label(root, text="🔈 Status: Stopped", bg="lightblue", font=("Arial", 12))
status_label.pack(pady=5)

# Volume control
def set_volume(val):
    volume = float(val)
    pygame.mixer.music.set_volume(volume)

volume_label = tk.Label(root, text="🔊 Volume", bg="lightblue", font=("Arial", 10))
volume_label.pack()
volume_slider = tk.Scale(root, from_=0, to=1, resolution=0.01, orient=tk.HORIZONTAL, command=set_volume, length=200)
volume_slider.set(0.5)
volume_slider.pack()

# Functions
def load_songs():
    folder = filedialog.askdirectory(title="Select Music Folder")
    if folder:
        listbox.delete(0, tk.END)
        song_list.clear()
        for file in os.listdir(folder):
            if file.lower().endswith(supported_formats):
                full_path = os.path.join(folder, file)
                song_list.append(full_path)
                listbox.insert(tk.END, file)
        if song_list:
            status_label.config(text=f"📁 Loaded {len(song_list)} song(s)")
        else:
            status_label.config(text="⚠️ No audio files found.")

def load_single_song():
    file_path = filedialog.askopenfilename(
        title="Select a Song",
        filetypes=[("Audio Files", "*.mp3 *.wav *.ogg *.flac")]
    )
    if file_path:
        listbox.delete(0, tk.END)
        song_list.clear()
        song_list.append(file_path)
        listbox.insert(tk.END, os.path.basename(file_path))
        status_label.config(text=f"🎵 Loaded: {os.path.basename(file_path)}")

def play_selected():
    global current_index
    selected = listbox.curselection()
    if selected:
        current_index = selected[0]
        play_song(current_index)
    else:
        messagebox.showwarning("Select a Song", "Please select a song to play.")

def play_song(index):
    global current_index, paused
    paused = False
    current_index = index
    song_path = song_list[index]
    pygame.mixer.music.load(song_path)
    pygame.mixer.music.set_volume(volume_slider.get())
    pygame.mixer.music.play()

    listbox.select_clear(0, tk.END)
    listbox.select_set(index)
    listbox.activate(index)

    song_name = os.path.basename(song_path)
    loop_text = " 🔁 Loop: ON" if looping else ""
    status_label.config(text=f"▶️ Playing: {song_name}{loop_text}")

    # Album Art
    try:
        audio = MP3(song_path, ID3=ID3)
        for tag in audio.tags.values():
            if tag.FrameID == "APIC":
                album_art_data = tag.data
                img = Image.open(io.BytesIO(album_art_data))
                img = img.resize((150, 150), Image.ANTIALIAS)
                photo = ImageTk.PhotoImage(img)
                img_label.config(image=photo)
                img_label.image = photo
                break
        else:
            raise Exception("No album art found")
    except Exception as e:
        img_label.config(image="", text="🎵", font=("Arial", 40))

def pause_resume():
    global paused
    if paused:
        pygame.mixer.music.unpause()
        paused = False
        if current_index is not None:
            song_name = os.path.basename(song_list[current_index])
            loop_text = " 🔁 Loop: ON" if looping else ""
            status_label.config(text=f"▶️ Resumed: {song_name}{loop_text}")
    else:
        pygame.mixer.music.pause()
        paused = True
        status_label.config(text="⏸️ Paused")

def stop_song():
    pygame.mixer.music.stop()
    status_label.config(text="🔈 Stopped")

def next_song():
    global current_index
    if current_index is not None and current_index < len(song_list) - 1:
        current_index += 1
        play_song(current_index)

def prev_song():
    global current_index
    if current_index is not None and current_index > 0:
        current_index -= 1
        play_song(current_index)

def toggle_loop():
    global looping
    looping = not looping
    loop_btn.config(text="🔁 Loop: ON" if looping else "🔁 Loop: OFF")
    if current_index is not None:
        song_name = os.path.basename(song_list[current_index])
        status_label.config(
            text=f"▶️ Playing: {song_name} 🔁 Loop: {'ON' if looping else 'OFF'}"
        )
    else:
        status_label.config(
            text="🔁 Looping Enabled" if looping else "🔁 Looping Disabled"
        )

def check_end():
    if not pygame.mixer.music.get_busy() and not paused:
        if looping and current_index is not None:
            play_song(current_index)
        elif current_index is not None and current_index < len(song_list) - 1:
            next_song()
    root.after(1000, check_end)

# Stop music on window close
def on_closing():
    pygame.mixer.music.stop()
    root.destroy()

# Buttons
loop_btn = tk.Button(root, text="🔁 Loop: OFF", command=toggle_loop, width=20, bg="white")
loop_btn.pack(pady=5)

tk.Button(root, text="⏮️ Previous", command=prev_song, width=20, bg="white").pack(pady=5)
tk.Button(root, text="⏯️ Pause/Resume", command=pause_resume, width=20, bg="white").pack(pady=5)
tk.Button(root, text="⏭️ Next", command=next_song, width=20, bg="white").pack(pady=5)

tk.Button(root, text="📂 Load Songs Folder", command=load_songs, width=20, bg="white").pack(pady=10)
tk.Button(root, text="🎵 Load Single Song", command=load_single_song, width=20, bg="white").pack(pady=5)

tk.Button(root, text="▶️ Play Selected", command=play_selected, width=20, bg="white").pack(pady=5)
tk.Button(root, text="⏹️ Stop", command=stop_song, width=20, bg="white").pack(pady=5)

# Song list
listbox = Listbox(root, width=50)
listbox.pack(pady=10)

# Run end checker
check_end()

# Handle exit
root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()
